In [1]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Dropping Row Number, CustomerId, Surname
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Gender has two values, so converting into numerical from categorical using Label Encoder.
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender']) # as the size of resultant matrix doesn't change. Saved in the same column.
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
onehot_encoder_geography = OneHotEncoder()
onehot_encoder_geo = onehot_encoder_geography.fit_transform(data[['Geography']]) ## 1Xn size is changed to 3Xn so saved in other data.
onehot_encoder_geo.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
onehot_encoder_geography.get_feature_names_out(['Geography']) ## get the list of the features that are built new

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_df = pd.DataFrame(onehot_encoder_geo.toarray(), columns=onehot_encoder_geography.get_feature_names_out(['Geography']))
geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [8]:
data = pd.concat([data.drop(['Geography'],axis=1),geo_df],axis=1)

In [9]:
X = data.drop(['EstimatedSalary'], axis=1)
y = data['EstimatedSalary']

In [10]:
## Split the data into train and test 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [11]:
## Apply scaling to the data 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [12]:
## save encoders in pickle file.
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geography.pkl','wb') as file:
    pickle.dump(onehot_encoder_geography, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

#### ANN Regression Implementaton

In [13]:
import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [14]:
## Build ANN Model
model = Sequential([
    Dense(64, activation='relu',input_shape = (X_train.shape[1],)), ## HL1 connected with input layer. Input size has to be applied for first HL.
    Dense(32, activation='relu'), # HL2
    Dense(1) ## output layer ## Default logistic regression will be considered so no need to provide activation function

])

c:\code\Gen AI Learning\ANN E2E Regression Project\ann_regression_venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
## compile the model or Optimize the model
# opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01) ## flexible learning rate
# loss = tensorflow.keras.losses.mae()
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=['mae'])


In [18]:
## create a directory for logs
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "regressionlogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [19]:
## Set up Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=15,restore_best_weights=True)

In [20]:
### Train the model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs =100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100310.5625 - mae: 100310.5625 - val_loss: 99584.8672 - val_mae: 99584.8672
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 99401.9141 - mae: 99401.9141 - val_loss: 98530.9844 - val_mae: 98530.9844
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 98057.6641 - mae: 98057.6641 - val_loss: 95812.7734 - val_mae: 95812.7734
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 94935.2969 - mae: 94935.2969 - val_loss: 91120.6641 - val_mae: 91120.6641
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 90470.6406 - mae: 90470.6406 - val_loss: 84654.7188 - val_mae: 84654.7188
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 82677.0234 - mae: 82677.0234 - val_loss: 77154.3125 - val_mae: 77154.3125
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 75262.8828 - mae: 75262.8828 - val_loss: 69630.6016 - val_mae: 69630.6016
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

In [21]:
model.save('model.h5')

In [22]:
## load tensorBoard Extension
%load_ext tensorboard

In [24]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 34476), started 0:00:08 ago. (Use '!kill 34476' to kill it.)